In [65]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [66]:
reviews = []
with open('data/yelp_academic_dataset_review.json', encoding="ansi") as fl:
    i=0
    for review in fl:
        reviews.append(json.loads(review))
        i+=1
        if i + 1 > 200000:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())

                review_id                 user_id             business_id  \
0  Q1sbwvVQXV2734tPgoKj4Q  hG7b0MtEbXx5QzbzE6C_VA  ujmEBvifdJM6h6RLv4wQIg   
1  GJXCdrto3ASJOqKeVWPi6Q  yXQM5uF2jS6es16SJzNHfg  NZnhc2sEQy3RmzKTZnqtwQ   
2  2TzJjDVDEuAW6MR5Vuc1ug  n6-Gk65cPZL6Uz8qRm3NYw  WTqjgwHlXbSFevF32_DJVw   
3  yi0R0Ugj_xUx_Nek0-_Qig  dacAIZ6fTM6mqwW5uxkskg  ikCg8xy5JIg_NGPx-MSIDA   
4  11a8sVPMUFtaC7_ABRkmtw  ssoyf2_x0EQMed6fgHeMyQ  b1b1eb3uo-w561D0ZfCEiQ   

   stars  useful  funny  cool  \
0    1.0       6      1     0   
1    5.0       0      0     0   
2    5.0       3      0     0   
3    5.0       0      0     0   
4    1.0       7      0     0   

                                                text                 date  
0  Total bill for this horrible service? Over $8G...  2013-05-07 04:34:36  
1  I *adore* Travis at the Hard Rock's new Kelly ...  2017-01-14 21:30:33  
2  I have to say that this office really has it t...  2016-11-09 20:09:03  
3  Went in for a lunch. Steak sand

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [67]:
business = []
with open('data/yelp_academic_dataset_business.json', encoding="ansi") as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())

              business_id                             name  \
0  1SWheh84yJXfytovILXOAQ       Arizona Biltmore Golf Club   
1  QXAEGFB4oINsVuTFxEYKFQ       Emerald Chinese Restaurant   
2  gnKjwL_1w79qoiV3IC_xQQ      Musashi Japanese Restaurant   
3  xvX2CttrVhyG2z1dFg_0xw  Farmers Insurance - Paul Lorenz   
4  HhyxOkGAM07SRYtlQ4wMFQ              Queen City Plumbing   

                          address         city state postal_code   latitude  \
0     2818 E Camino Acequia Drive      Phoenix    AZ       85016  33.522143   
1            30 Eglinton Avenue W  Mississauga    ON     L5R 3E7  43.605499   
2       10110 Johnston Rd, Ste 15    Charlotte    NC       28210  35.092564   
3   15655 W Roosevelt St, Ste 237     Goodyear    AZ       85338  33.455613   
4  4209 Stuart Andrew Blvd, Ste F    Charlotte    NC       28217  35.190012   

    longitude  stars  review_count  is_open  \
0 -112.018481    3.0             5        0   
1  -79.652289    2.5           128        1   
2  -80.8591

ELIMINAMOS LAS COLUMNAS INNECESARIAS

In [68]:
df_review = df_review.drop(["review_id", "user_id", "useful", "funny", "cool", "date"], axis=1)
df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

COMPROBAMOS

In [69]:
print(df_review.head())
print(df_business.head())

              business_id  stars  \
0  ujmEBvifdJM6h6RLv4wQIg    1.0   
1  NZnhc2sEQy3RmzKTZnqtwQ    5.0   
2  WTqjgwHlXbSFevF32_DJVw    5.0   
3  ikCg8xy5JIg_NGPx-MSIDA    5.0   
4  b1b1eb3uo-w561D0ZfCEiQ    1.0   

                                                text  
0  Total bill for this horrible service? Over $8G...  
1  I *adore* Travis at the Hard Rock's new Kelly ...  
2  I have to say that this office really has it t...  
3  Went in for a lunch. Steak sandwich was delici...  
4  Today was my second out of three sessions I ha...  
              business_id                                         categories
0  1SWheh84yJXfytovILXOAQ                                  Golf, Active Life
1  QXAEGFB4oINsVuTFxEYKFQ  Specialty Food, Restaurants, Dim Sum, Imported...
2  gnKjwL_1w79qoiV3IC_xQQ                  Sushi Bars, Restaurants, Japanese
3  xvX2CttrVhyG2z1dFg_0xw                      Insurance, Financial Services
4  HhyxOkGAM07SRYtlQ4wMFQ  Plumbing, Shopping, Local Services, Home 

Preprocesado a lo fácil para poder trabajar con ello

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

vectorizer = TfidfVectorizer()
lemma = WordNetLemmatizer()

X = []
for r in df_review["text"]:
    txt = word_tokenize(r.lower())
    newtxt = ""
    for w in txt:
        l = lemma.lemmatize(w)
        newtxt += l+" "
    X.append(newtxt)

X = vectorizer.fit_transform(X)

Lo que estoy haciendo realmente, hay que terminarlo pero con la opción de encima va bien

In [71]:
'''from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
bag = {}  #Menciones totales de cada palabra
freq = {} #Menciones de cada palabra en cada documento
length = {}  #Número de palabras de cada documento
count = {}  #Número de documentos que contienen cada la palabra
lemma = WordNetLemmatizer()
for r in df_review["text"]:
    txt = word_tokenize(r.lower())
    length[r] = len(txt)
    checked = []  #Palabras del documento ya introducidas en count
    freqs = {}  #Menciones de cada palabra en el documento
    for w in txt:
        l = lemma.lemmatize(w)
        if not l in count:
            count[l] = 0
        if not l in checked:
            count[l] += 1
            checked.append(l)
        if not l in bag:
            bag[l] = 0
        bag[l] += 1
        if not l in freqs:
            freqs[l] = 0
        freqs[l] += 1
    freq[r] = freqs
print(sum(length.values())/len(length.values()))
print(len(bag))'''

'from nltk.tokenize import word_tokenize\nfrom nltk.stem import WordNetLemmatizer\nbag = {}  #Menciones totales de cada palabra\nfreq = {} #Menciones de cada palabra en cada documento\nlength = {}  #Número de palabras de cada documento\ncount = {}  #Número de documentos que contienen cada la palabra\nlemma = WordNetLemmatizer()\nfor r in df_review["text"]:\n    txt = word_tokenize(r.lower())\n    length[r] = len(txt)\n    checked = []  #Palabras del documento ya introducidas en count\n    freqs = {}  #Menciones de cada palabra en el documento\n    for w in txt:\n        l = lemma.lemmatize(w)\n        if not l in count:\n            count[l] = 0\n        if not l in checked:\n            count[l] += 1\n            checked.append(l)\n        if not l in bag:\n            bag[l] = 0\n        bag[l] += 1\n        if not l in freqs:\n            freqs[l] = 0\n        freqs[l] += 1\n    freq[r] = freqs\nprint(sum(length.values())/len(length.values()))\nprint(len(bag))'

In [72]:
'''import math
list = {}
for i in range(0, len(length)-1):
    txt = df_review["text"][i].lower()
    for w in bag.keys():
        tf = freq[txt][w]/length[txt]
        idf = math.log10(len(length)/count[w])
        if not w in list:
            list[w] = []
        list[w].append(tf*idf)
for w in bag.keys():
    df_review[w] = list'''

'import math\nlist = {}\nfor i in range(0, len(length)-1):\n    txt = df_review["text"][i].lower()\n    for w in bag.keys():\n        tf = freq[txt][w]/length[txt]\n        idf = math.log10(len(length)/count[w])\n        if not w in list:\n            list[w] = []\n        list[w].append(tf*idf)\nfor w in bag.keys():\n    df_review[w] = list'

In [74]:
print(X)

  (0, 22904)	0.1373833046763468
  (0, 6009)	0.0666710987458892
  (0, 8801)	0.06042381731209967
  (0, 31673)	0.22324645070650057
  (0, 43453)	0.19642763884648456
  (0, 9526)	0.16139706624695224
  (0, 29714)	0.11916114244009669
  (0, 17875)	0.1944735485655492
  (0, 907)	0.19862444430735715
  (0, 11105)	0.06147522731027972
  (0, 16546)	0.08008879733556
  (0, 62412)	0.15267243428586294
  (0, 18625)	0.15145333303193056
  (0, 66778)	0.49478172133310416
  (0, 3071)	0.24739086066655208
  (0, 18385)	0.13733438652720664
  (0, 88625)	0.03745306283856678
  (0, 60116)	0.2193646113173283
  (0, 87539)	0.06419157089647097
  (0, 40811)	0.11443690830471603
  (0, 4679)	0.1182682302524849
  (0, 23829)	0.25447431968020345
  (0, 87729)	0.11571470303354955
  (0, 3573)	0.36236867784570664
  (0, 63398)	0.0920406734672639
  :	:
  (199999, 82448)	0.1349482381597636
  (199999, 83109)	0.06407328348191806
  (199999, 9908)	0.05264951177492185
  (199999, 4737)	0.10455735200612472
  (199999, 42248)	0.04938134089706876